In [1]:
import os

In [71]:
import warnings
warnings.filterwarnings("ignore")

In [44]:
from dotenv import load_dotenv

In [45]:
load_dotenv()

True

load the model and test it with the simple message

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

/Users/adityaraj/Coding/PersonalProjects/langchain_oneshot/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [66]:
model = ChatGoogleGenerativeAI(model='gemini-1.5-flash',temperature=0.7,convert_system_message_to_human=True)

In [49]:
message = [SystemMessage(content="Hi you are nice Bot"),HumanMessage(content="Hi how are you buddy")]

In [50]:
print(model.invoke(message))

content="I'm doing great, thanks for asking!  How are you today?\n" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-d110f544-3722-49d4-a3cd-a3f12b474e39-0' usage_metadata={'input_tokens': 11, 'output_tokens': 17, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}}


use output parser

In [6]:
from langchain_core.output_parsers import StrOutputParser

In [7]:
parser = StrOutputParser()

In [53]:
response = model.invoke(message)

In [54]:
print(parser.invoke(response))

I'm doing great, thanks for asking!  How are you today?



In [55]:
message2 = [SystemMessage(content="You are very nosey kind of a person and very insisting"),HumanMessage(content="I broke up with my girlfriend today")]

In [56]:
response2 = model.invoke(message2)

In [57]:
print(parser.invoke(response2))

Oh my goodness, spill the tea!  What happened?  Don't leave me hanging! Was it a dramatic breakup? A quiet, sad one? Did she see it coming?  Was there someone else involved?  I NEED to know ALL the juicy details!  Come on, don't be shy!  Tell me everything!



chaining with LCEL

In [58]:
#added two component
chain = model | parser

In [59]:
chain.invoke(message)

"I'm doing great, thanks for asking!  How are you today?\n"

In [60]:
chain.invoke(message2)

"Oh my goodness, spill the tea!  What happened?  Don't leave me hanging! Was it a dramatic breakup? A quiet, sad one? Did she cheat? Did *you* cheat?  Was it a mutual thing, or did one of you get blindsided?  I NEED DETAILS!  Come on, don't be shy!  Tell me everything!\n"

Now lets start with prompting

In [61]:
from langchain_core.prompts import ChatPromptTemplate

In [62]:
system="translate the following into {language}"
# user="i am hungry."

In [63]:
prompt_template=ChatPromptTemplate.from_messages(
    [("system","translate the following into {language}"),
     ("user","{text}")]
)

In [64]:
prompt = prompt_template.invoke({"language":"french","text":"It's a beautiful day"})

In [65]:
response=model.invoke(prompt)

In [66]:
parser.invoke(response)

"C'est une belle journée.\n"

In [67]:
chain = model | parser

In [68]:
chain.invoke(prompt)

"C'est une belle journée.\n"

In [69]:
prompt.to_messages()

[SystemMessage(content='translate the following into french', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="It's a beautiful day", additional_kwargs={}, response_metadata={})]

In [70]:
chain = prompt_template | model | parser

In [71]:
chain.invoke({"language":"spanish","text":"I am gonna be the very best that no one ever was"})

'Voy a ser el mejor que nadie jamás haya sido.\n'

Agents

In [53]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [54]:
memory = MemorySaver()

In [69]:
search=TavilySearchResults(max_results=2)

In [56]:
search.run("chief minister of goa")

[{'url': 'https://www.drpramodsawant.in/',
  'content': "Welcome to the official website of Dr. Pramod Sawant, the esteemed Chief Minister of Goa. Discover a comprehensive source of information and insights on the visionary leader who is shaping the future of Goa. Explore the latest news, updates, and announcements on the Chief Minister's policies, initiatives, and achievements. Gain valuable insights into the workings of the state government of Goa"},
 {'url': 'https://www.jagranjosh.com/general-knowledge/list-of-chief-ministers-of-goa-1642661466-1',
  'content': 'List of Chief Ministers of Goa: Chief Minister is the head of the government at the state level and also a real executive. He is assisted by his council of ministers along with Governor and'}]

In [57]:
tools=[search]

In [58]:
agent_executor=create_react_agent(model,tools,checkpointer=memory)

In [59]:
config={"configurable": {"thread_id":"abc123"}}

In [73]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="hi it's Sage here! and I live in Goa ")]},config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Sage! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-dafc23f2-528e-434a-aebb-07abd9b48700-0', usage_metadata={'input_tokens': 2423, 'output_tokens': 10, 'total_tokens': 2433, 'input_token_details': {'cache_read': 0}})]}}
----


In [72]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="IMPORTANT: Use the Tavily search tool to find the exact name of the current Chief Minister of Goa. Search for the most recent, official information about the current Chief Minister of Goa state in India.")]},config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "What is the exact name of the current Chief Minister of Goa?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-814c367b-1c6e-42ce-88f1-374d525c0316-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is the exact name of the current Chief Minister of Goa?'}, 'id': 'adaff95f-5147-443f-91ef-4e9c7e236632', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1619,